Import packages

In [1]:
import pandas as pd
from cassandra.cluster import Cluster

Create session connection to cassandra

In [2]:
clstr = Cluster()
session = clstr.connect()

Use session to talk to cassandra


In [3]:
session.execute("DROP KEYSPACE IF EXISTS m14")

In [7]:
session.execute("DROP KEYSPACE IF EXISTS w04python")

In [8]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


In [9]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':3
    }
""")

In [10]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


Create a new table for inventory

In [11]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14.inventory (
    sku TEXT, 
    name TEXT, 
    description TEXT, 
    warehouse_num INT,
    PRIMARY KEY(sku, warehouse_num)
);
""")

LOAD THE DATA

In [12]:
warehouse_data = [
    ("RRY001", "REFRIGIRATOR", "whirlpool", 9),
    ("RRY002", "Mattress", "zinus", 7),
    ("RRY003", "laptop", "lenovo", 5),
    ("RRY004", "camera", "SONY", 4),
    ("RRY005", "shoes", "NIKE JORDAN", 3),
    ("RRY006", "pillow", "zinus", 2),
    ("RRY007", "headphones", "Apple Airpods", 1),
    ("RRY008", "coffee", "Starbucks", 4),
    ("RRY009", "Tyres", "Apollo", 11),
    ("RRY010", "Air Conditioner", "Llyod", 7)
]
for entry in warehouse_data:
    insert_query = """
    INSERT INTO m14.inventory (sku, name, description, warehouse_num)
    VALUES (%s, %s, %s, %s)
    """
    session.execute(insert_query, entry)

    

QUERY TO FETCH DATA

In [13]:

query = "SELECT * FROM m14.inventory"
rows = session.execute(query)

# Print the data
print("sku\t\tname\t\tdescription\t\twarehouse_num")
print("-" * 80)  # just a separator line
for row in rows:
    print(f"{row.sku}\t\t{row.name}\t\t{row.description}\t\t{row.warehouse_num}")


sku		name		description		warehouse_num
--------------------------------------------------------------------------------
RRY004		camera		SONY		4
RRY006		pillow		zinus		2
RRY007		headphones		Apple Airpods		1
RRY008		coffee		Starbucks		4
RRY010		Air Conditioner		Llyod		7
RRY005		shoes		NIKE JORDAN		3
RRY002		Mattress		zinus		7
RRY003		laptop		lenovo		5
RRY001		REFRIGIRATOR		whirlpool		9
RRY009		Tyres		Apollo		11


In [14]:
rows = session.execute("SELECT (sku, name, description, warehouse_num) FROM m14.inventory WHERE warehouse_num = 7 ALLOW FILTERING")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

RRY010, Air Conditioner, Llyod, 7
RRY002, Mattress, zinus, 7


INDEXING

In [15]:
session.execute("""
CREATE INDEX IF NOT EXISTS warehouse_num
ON m14.inventory (warehouse_num);
""")

In [16]:
rows = session.execute("SELECT (sku, name, description, warehouse_num) FROM m14.inventory WHERE warehouse_num = 1 ")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

RRY007, headphones, Apple Airpods, 1
